In [ ]:
import requests
import pandas as pd

In [ ]:
# Load the CSV file with scraped condo names
df = pd.read_csv(r"condoNames.csv")

In [ ]:
# Define the API endpoint and parameters
url = 'https://developers.onemap.sg/commonapi/search'
params = {
    'returnGeom': 'Y',
    'getAddrDetails': 'Y',
    'pageNum': 1
}

In [ ]:
# Initialize an empty list to store the results
results = []

In [ ]:
# Iterate over each condo name in the 'df' dataframe
for Name in df['Name']:
    # Set the API parameters
    params['searchVal'] = Name
    
    # Send a GET request to the OneMap API
    response = requests.get(url, params=params)
    
    # Parse the JSON response
    data = response.json()
    
    # Check if the response contains any results
    if 'results' in data and len(data['results']) > 0:
        # Extract the first result (assuming it's the most relevant)
        first_result = data['results'][0]
        
        # Extract the postal code, latitude, and longitude from the result
        postal_code = first_result['POSTAL']
        latitude = first_result['LATITUDE']
        longitude = first_result['LONGITUDE']
        
        # Check if postal code is "NIL"
        if postal_code == "NIL":
            # If postal code is "NIL", try to retrieve the second result
            if len(data['results']) > 1:
                second_result = data['results'][1]
                postal_code = second_result['POSTAL']
                latitude = second_result['LATITUDE']
                longitude = second_result['LONGITUDE']
        
        # Append the results to the list
        results.append({
            'name': Name,
            'postal': postal_code,
            'lat': latitude,
            'long': longitude
        })
    else:
        # Append None values to the list when no results are found
        results.append({
            'name': Name,
            'postal': None,
            'lat': None,
            'long': None
        })

In [ ]:
# Create a dataframe from the results list using pd.concat
results_df = pd.concat([pd.DataFrame([result]) for result in results], ignore_index=True)

In [ ]:
# Save the results to a CSV file
results_df.to_csv('condo_results.csv', index=False)